## Imports

In [ ]:
import datetime
import os

In [ ]:
from sklearn.model_selection import StratifiedKFold

In [ ]:
from keras import backend as K
from keras.models import Sequential
from keras.layers import *
from keras.callbacks import EarlyStopping, ModelCheckpoint

## Config

In [ ]:
feature_list_id = 'oofp_nn_concat_dense_1'

In [ ]:
RANDOM_SEED = 42

In [ ]:
np.random.seed(RANDOM_SEED)

In [ ]:
data_folder = os.path.abspath(os.path.join(os.curdir, os.pardir, 'data')) + os.path.sep
aux_data_folder = os.path.join(data_folder, 'aux') + os.path.sep
preproc_data_folder = os.path.join(data_folder, 'preproc') + os.path.sep
features_data_folder = os.path.join(data_folder, 'features') + os.path.sep
submissions_data_folder = os.path.join(data_folder, 'submissions') + os.path.sep

## Read Data

In [ ]:
embedding_matrix = load(aux_data_folder + 'embedding_weights_fasttext_filtered_no_stopwords.pickle')

In [ ]:
X_train_q1 = load(features_data_folder + 'X_train_nn_fasttext_q1_filtered_no_stopwords.pickle')
X_train_q2 = load(features_data_folder + 'X_train_nn_fasttext_q2_filtered_no_stopwords.pickle')

In [ ]:
X_train = np.stack((X_train_q1, X_train_q2), axis=1)

In [ ]:
y_train = load(features_data_folder + 'y_train.pickle')

In [ ]:
EMBEDDING_DIM = embedding_matrix.shape[-1]
VOCAB_LENGTH = embedding_matrix.shape[0] - 1
MAX_SEQUENCE_LENGTH = X_train.shape[-1]

In [ ]:
print(EMBEDDING_DIM, VOCAB_LENGTH, MAX_SEQUENCE_LENGTH)

## Train Models & Compute Out-of-Fold Predictions

In [ ]:
def create_model():
    model_q1 = Sequential()

    model_q1.add(Embedding(
        VOCAB_LENGTH + 1,
        EMBEDDING_DIM,
        weights=[embedding_matrix],
        input_length=MAX_SEQUENCE_LENGTH,
        trainable=False,
    ))

    model_q1.add(TimeDistributed(Dense(
        EMBEDDING_DIM,
        activation='relu',
    )))

    model_q1.add(Lambda(lambda x: K.max(x, axis=1), output_shape=(EMBEDDING_DIM, )))

    model_q2 = Sequential()

    model_q2.add(Embedding(
        VOCAB_LENGTH + 1,
        EMBEDDING_DIM,
        weights=[embedding_matrix],
        input_length=MAX_SEQUENCE_LENGTH,
        trainable=False,
    ))

    model_q2.add(TimeDistributed(Dense(
        EMBEDDING_DIM,
        activation='relu'
    )))

    model_q2.add(Lambda(lambda x: K.max(x, axis=1), output_shape=(EMBEDDING_DIM, )))

    model = Sequential()
    model.add(Merge([model_q1, model_q2], mode='concat'))

    model.add(BatchNormalization())
    model.add(Dense(200, activation='relu'))

    model.add(BatchNormalization())
    model.add(Dense(200, activation='relu'))

    model.add(BatchNormalization())
    model.add(Dense(200, activation='relu'))

    model.add(BatchNormalization())
    model.add(Dense(200, activation='relu'))

    model.add(BatchNormalization())
    model.add(Dense(1, activation='sigmoid'))

    model.compile(
        loss='binary_crossentropy', 
        optimizer='adam', 
        metrics=['accuracy']
    )

    return model

In [ ]:
model_checkpoint_path = aux_data_folder + 'fold-checkpoint-' + feature_list_id + '.h5'

In [ ]:
kfold = StratifiedKFold(
    n_splits=5,
    shuffle=True,
    random_state=RANDOM_SEED
)

In [ ]:
y_train_oofp = np.zeros_like(y_train, dtype='float64')

In [ ]:
%%time

for fold_num, (ix_fold_train, ix_fold_val) in enumerate(kfold.split(X_train, y_train)):
    X_fold_train = X_train[ix_fold_train]
    y_fold_train = y_train[ix_fold_train]
    
    X_fold_val = X_train[ix_fold_val]
    y_fold_val = y_train[ix_fold_val]
    
    print()
    print(f'Fitting fold {fold_num + 1} of {kfold.n_splits}')
    print()
    
    model = create_model()
    model.fit(
        [X_fold_train[:, 0], X_fold_train[:, 1]], y_fold_train,
        batch_size=64,
        epochs=200,
        validation_data=([X_fold_val[:, 0], X_fold_val[:, 1]], y_fold_val),
        class_weight=keras_get_class_weights(y_fold_train),
        callbacks=[
            EarlyStopping(
                monitor='val_loss',
                min_delta=0.005,
                patience=3,
                verbose=1,
                mode='auto',
            ),
            ModelCheckpoint(
                model_checkpoint_path,
                monitor='val_loss',
                save_best_only=True,
                verbose=2,
            ),
        ],
    )
        
    # Create out-of-fold prediction.
    model.load_weights(model_checkpoint_path)
    y_pred_oofp = model.predict_proba([X_fold_val[:, 0], X_fold_val[:, 1]])[:, -1]
    
    # Remember them.
    y_train_oofp[ix_fold_val] = y_pred_oofp

## Save feature names

In [ ]:
feature_names = [
    'oofp_nn_concat_dense_1',
]

In [ ]:
save_lines(feature_names, features_data_folder + f'X_train_{feature_list_id}.names')

### Save Train features

In [ ]:
y_train_oofp = y_train_oofp.reshape((-1, 1))

In [ ]:
save(y_train_oofp, features_data_folder + f'X_train_{feature_list_id}.pickle')

## Save Test features

In [ ]:
X_test_q1 = load(features_data_folder + 'X_test_nn_fasttext_q1_filtered_no_stopwords.pickle')
X_test_q2 = load(features_data_folder + 'X_test_nn_fasttext_q2_filtered_no_stopwords.pickle')

In [ ]:
# It would be better to fit the model on the whole training set
# but the validation set for early stopping would be an issue.
y_test_oofp = model.predict_proba([X_test_q1, X_test_q2])[:, -1]

In [ ]:
y_test_oofp = y_test_oofp.reshape((-1, 1))

In [ ]:
save(y_test_oofp, features_data_folder + f'X_test_{feature_list_id}.pickle')